In [ ]:
import functools

import numpy as np
import tensorflow as tf

In [ ]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

In [ ]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

In [ ]:
!head {train_file_path}

In [ ]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)
print(raw_train_data)

In [ ]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))
show_batch(raw_train_data)
# 这里可以看到的是，虽然batch了，但batch的结果有点类似于对tuple的batch
# data里的每个元素都是一个OrderedDict，k代表着字段名，而v是一个形状为(None,)的数组，这个None就是batch的结果。

In [ ]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

In [ ]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

In [ ]:
print(temp_dataset)
example_batch, labels_batch = next(iter(temp_dataset)) 
print(example_batch)
print(labels_batch)
# temp_dataset持有的是orderDict

In [ ]:
def pack(features, label):
  print(features)# 可以看到这个feature直接就是order dict，他的values才是tensor，所以不用挂py_function
  print(features.values())
  return tf.stack(list(features.values()), axis=-1), label
packed_dataset = temp_dataset.map(pack)

# 所以说，map的时候，pack函数就会执行，但实际上只会执行一次，但这个执行并不是真正意义上的计算，本质是将一个op放在graph上。
# 所以对张量取值是取不到的
# 所以什么时候需要包py_function呢，只要涉及对tensor的具体值的处理，就需要包
# 但就像这个例子里，temp_dataset里持有的是orderdict，并不是tensor，所以不用包
# 再例如如果一个dataset里持有的是tuple，只要不涉及对tensor具体值的处理，也不用包
# 就像ds.map(lambda x,y:(1,2))，你看，这也不用py_function
for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

In [ ]:
show_batch(raw_train_data)

In [ ]:
print(raw_train_data)
example_batch, labels_batch = next(iter(raw_train_data)) 
print(example_batch)
print(labels_batch)

In [ ]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [ ]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']
# 就是普通的map操作，不要多想，上面的call就是一个function，features就是orderdict
packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))
example_batch, labels_batch = next(iter(packed_train_data)) 

In [ ]:
show_batch(packed_train_data)
print(example_batch)
print(labels_batch)

In [ ]:
!pip install -U pandas
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

In [ ]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [ ]:
MEAN

In [ ]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)# 构造了一个偏函数，骚，但没必要

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column
# 定义了一个列对象，注意这个numeric_column的第一个参数key就是numeric，对应的就是data里每个orderdict里的numeric这个key

In [ ]:
example_batch['numeric']

In [ ]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
# 把这个feature_column作为对象传给了densefeature这一层
# 这一层会对输入的dict进行处理，会将这个orderdict里的numeric对应的value进行处理。
# 但实际上= =em。。。每个order dict实际上就是一个batch，所以这个layer可以直接被扔进model里
numeric_layer(example_batch).numpy()

In [ ]:
numeric_layer(packed_train_data).numpy() 
# 报错
# We expected a dictionary here. Instead we got，说明只能处理dict

In [ ]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))
    
print(categorical_columns)

In [ ]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy())# 差不多和上面类似，就是进行one hot

In [ ]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)
print(preprocessing_layer(example_batch).numpy())
# 好活

In [ ]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1),
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data
model.fit(train_data, epochs=20)
predictions = model.predict(test_data)

# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  prediction = tf.sigmoid(prediction).numpy()
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))
# 补充一句，model这个对象的predict evaluate 都处理的是dataset
# 但其__call__处理的是其中的一个batch
# 就和其持有的各个layer一样，每次也只能处理一个batch，例如数字列表等等

In [ ]:
packed_train_data

In [ ]:
import numpy as np
import tensorflow as tf
print("==========================")
from collections import OrderedDict
import numpy as np
tf.v
def get_dict(i):
  d=OrderedDict()
  d["a"]=tf.cast(i,tf.float32)
  d["b"]=np.random.normal()
  return (d,i)

test_ds=tf.data.Dataset.range(1,4).map(lambda x:get_dict(x))
print(test_ds)

def pack(features, label):
  print("print once only",features)# 可以看到这个feature直接就是order dict，他的values才是tensor，所以不用挂py_function
  values=list(features.values())
  return tf.add(values,1.1),label
packed_dataset = test_ds.map(pack)

# 所以说，map的时候，pack函数就会执行，但实际上只会执行一次，但这个执行并不是真正意义上的计算，本质是将一个op放在graph上。
# 所以对张量取值是取不到的
# 所以什么时候需要包py_function呢，只要涉及对tensor的具体值的处理，就需要包
# 但就像这个例子里，temp_dataset里持有的是orderdict，并不是tensor，所以不用包
# 再例如如果一个dataset里持有的是tuple，只要不涉及对tensor具体值的处理，也不用包
# 就像ds.map(lambda x,y:(1,2))，你看，这也不用py_function
for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

In [ ]:
test_ds